In [ ]:
import torch
import torch.nn as nn
import json
import os
import speech_recognition as sr
from gtts import gTTS
import playsound
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# تعريف نموذج الشبكة العصبية
class ChatBotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatBotModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

# تحميل ملف البيانات (الذي يحتوي على الأسئلة والأجوبة)
with open(r"C:\Users\M L A\Desktop\data\data.json", encoding="utf-8") as f:
    data = json.load(f)

# إعداد المعالجة
questions = [item['input_text'] for item in data['responses']]
answers = [item['response_text'] for item in data['responses']]
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(answers)

vectorizer = CountVectorizer(max_features=50)
vectorizer.fit(questions)

# تحميل النموذج المدرب
input_size = 50  # عدد الميزات في الجمل
hidden_size = 8  # حجم الطبقة المخفية
output_size = len(set(encoded_labels))  # عدد الفئات المختلفة
model = ChatBotModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load(r"C:\Users\M L A\Desktop\data\chatbot_model1.pt", map_location=torch.device('cpu')))
model.eval()

# دالة لتحويل النص إلى صوت باستخدام gTTS
def speak_gtts(text, lang='ar'):
    """تحويل النص إلى صوت وتشغيله باستخدام مكتبة gTTS"""
    try:
        tts = gTTS(text=text, lang=lang)
        tts.save("response.mp3")  # حفظ الملف الصوتي مؤقتًا
        playsound.playsound("response.mp3")  # تشغيل الملف الصوتي
        os.remove("response.mp3")  # حذف الملف بعد تشغيله
    except Exception as e:
        print(f"حدث خطأ أثناء تحويل النص إلى صوت باستخدام gTTS: {e}")

# دالة للتنبؤ بالرد باستخدام النموذج
def predict_response(question):
    """استخدام النموذج المدرب للتنبؤ بالرد بناءً على السؤال"""
    question_vector = vectorizer.transform([question]).toarray()
    question_tensor = torch.tensor(question_vector, dtype=torch.float32)
    with torch.no_grad():
        output = model(question_tensor)
        _, predicted = torch.max(output, 1)
    return label_encoder.inverse_transform([predicted.item()])[0]

# دالة للاستماع والرد
def listen_and_respond():
    """الاستماع لصوت المستخدم وتحويله إلى نص والرد بناءً عليه"""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("تحدث الآن...")
        try:
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=7)
            text = recognizer.recognize_google(audio, language="ar")
            print("لقد قلت:", text)
            response = predict_response(text)  # استخدام النموذج لتحديد الرد
            print("رد:", response)
            speak_gtts(response)  # تحويل الرد إلى صوت
                
        except sr.WaitTimeoutError:
            print("لم تبدأ التحدث في الوقت المحدد.")
            speak_gtts("لم تبدأ التحدث في الوقت المحدد.")
        except sr.UnknownValueError:
            print("لم أتمكن من فهم الصوت.")
            speak_gtts("لم أتمكن من فهم الصوت.")
        except sr.RequestError:
            print("حدث خطأ في الاتصال بخدمة التعرف على الصوت.")
            speak_gtts("حدث خطأ في الاتصال بالخدمة.")

        return True  # الاستمرار في الاستماع بعد الرد

# تشغيل الروبوت في حلقة تفاعلية
while True:
    if not listen_and_respond():
        break  # إنهاء البرنامج عند وداع المستخدم
